In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv('labeled_data.csv')

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#\w+', '', tweet)
    tweet = re.sub(r'\d+', '', tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'\W', ' ', tweet)
    tweet = re.sub(r'\s+', ' ', tweet)
    return tweet

df['clean_tweet'] = df['tweet'].apply(clean_tweet)

In [ ]:
df['clean_tweet'] = df['tweet'].apply(clean_tweet)

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_tweet(tweet):
    words = tweet.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

In [ ]:
df['processed_tweet'] = df['clean_tweet'].apply(preprocess_tweet)

In [ ]:
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['processed_tweet'])
sequences = tokenizer.texts_to_sequences(df['processed_tweet'])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

In [ ]:
X = padded_sequences
y = df['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_reshaped, y_train)
X_train_resampled = X_train_resampled.reshape(X_train_resampled.shape[0], max_len)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)

In [ ]:
history = model.fit(X_train_resampled, y_train_resampled, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/20
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 276s 237ms/step - accuracy: 0.6395 - loss: 0.7949 - val_accuracy: 0.0817 - val_loss: 1.6752
Epoch 2/20
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 295s 213ms/step - accuracy: 0.7923 - loss: 0.5197 - val_accuracy: 0.2510 - val_loss: 1.4958
Epoch 3/20
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 243s 211ms/step - accuracy: 0.8428 - loss: 0.4123 - val_accuracy: 0.3338 - val_loss: 1.5100
Epoch 4/20
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 268s 217ms/step - accuracy: 0.8787 - loss: 0.3266 - val_accuracy: 0.3228 - val_loss: 1.5834
Epoch 5/20
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 259s 214ms/step - accuracy: 0.9058 - loss: 0.2629 - val_accuracy: 0.2920 - val_loss: 2.0153
Epoch 6/20
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 256s 209ms/step - accuracy: 0.9257 - loss: 0.2100 - val_accuracy: 0.2982 - val_loss: 2.3749
Epoch 7/20
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 261s 208ms/step - accuracy: 0.9400 - loss: 0.1664 - val_accuracy: 0.2927 - val_loss: 3.0507
Epoch 8/20
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 261s 207ms/step - ac

In [ ]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', average='weighted')

155/155 ━━━━━━━━━━━━━━━━━━━━ 10s 65ms/step


In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score (weighted): {f1:.4f}")
print(f"ROC AUC (weighted, ovo): {roc_auc:.4f}")

Accuracy: 0.8257
F1-score (weighted): 0.8477
ROC AUC (weighted, ovo): 0.8763


In [ ]:
def test_input(text):
    cleaned_text = clean_tweet(text)
    processed_text = preprocess_tweet(cleaned_text)
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    predicted_probabilities = model.predict(padded_sequence)[0]
    predicted_class = np.argmax(predicted_probabilities)
    class_labels = {0: 'Hate Speech', 1: 'Offensive Language', 2: 'Neither'}
    print(f"Input: {text}")
    print(f"Predicted Class: {class_labels[predicted_class]}")
    print("Class Probabilities:")
    for cls, prob in zip(class_labels.values(), predicted_probabilities):
        print(f"  {cls}: {prob:.4f}")

In [ ]:
example_text = "Stop, you are not ugly but so pretty"
test_input(example_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Input: Stop, you are not ugly but so pretty
Predicted Class: Hate Speech
Class Probabilities:
  Hate Speech: 0.8805
  Offensive Language: 0.0010
  Neither: 0.1186
